In [25]:
import requests
import csv
import time
from datetime import datetime


In [26]:
api_key = "a6275c028adf4e9bc5ae5f67edfb4c5f"

annee_debut = 1990
annee_fin = datetime.now().year

with open('films.csv', 'w', newline='') as csvfile:
    fieldnames = ['titre_fr', 'realisateur', 'acteurs', 'genres', 'budget', 'date_sortie', 'compagnies_production']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    for annee in range(annee_debut, annee_fin + 1):
        page = 1
        while True:
            response = requests.get(f"https://api.themoviedb.org/3/discover/movie?api_key={api_key}&primary_release_year={annee}&region=FR&page={page}&language=fr-FR")
            data = response.json()

            for film in data['results']:
                response = requests.get(f"https://api.themoviedb.org/3/movie/{film['id']}?api_key={api_key}&language=fr-FR")
                details = response.json()

                response = requests.get(f"https://api.themoviedb.org/3/movie/{film['id']}/credits?api_key={api_key}&language=fr-FR")
                credits = response.json()

                companies = details['production_companies']
                compagnies_production = [company['name'] for company in companies]

                realisateur = next((membre for membre in credits['crew'] if membre['job'] == 'Director'), None)
                if realisateur is not None:
                    realisateur = realisateur['name']

                acteurs = []
                for acteur in credits['cast']:
                    if acteur['order'] <= 3:
                        acteurs.append(acteur['name'])

                genres = [genre['name'] for genre in details['genres']]

                if details['budget'] > 1000000 :
                    writer.writerow({'titre_fr': details['title'], 'realisateur': realisateur, 'acteurs': acteurs, 'genres': genres, 'budget': details['budget'], 'date_sortie': film['release_date'], 'compagnies_production': compagnies_production})

            if page >= data['total_pages']:
                break

            page += 1
            
            

            time.sleep(0.10)


In [ ]:

total_pages = 0

with open('celebrite.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(['Célébrité', 'Type', 'Classement', 'Popularité'])
    
    page = 1
    while True:
        url = f'https://api.themoviedb.org/3/trending/person/week?api_key={api_key}&page={page}'
        
        response = requests.get(url)
        
        if response.status_code == 200:
            popular_people = response.json()['results']
            
            if len(popular_people) > 0:
                for index, person in enumerate(popular_people):
                    name = person['name']
                    media_type = person['media_type']
                    popularity = person['popularity']
                    rank = (page - 1) * 20 + index + 1
                    
                    if popularity >= 1.1:
                        writer.writerow([name, media_type, rank, popularity])
                
                page += 1
                total_pages += 1
                
                time.sleep(0.25)
            else:
                break